# Fitting distributions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
size = 100
xdata = np.linspace(0, 10, size) + np.random.normal(scale=0.01, size=size)
ydata = 3 * np.sin(xdata) + np.random.normal(scale=0.1, size=size)

In [ ]:
plt.figure()
plt.plot(xdata, ydata, 'o')

In [ ]:
def cos_func(x, yscale, xscale, xshift):
    y = yscale * np.cos(xscale * x + xshift)
    return y

In [ ]:
parameters, covariance = curve_fit(cos_func, xdata, ydata)

In [ ]:
fit_yscale = parameters[0]
fit_xscale = parameters[1]
fit_xshift = parameters[2]

fit_cosine = cos_func(xdata, fit_yscale, fit_xscale, fit_xshift)
# a better way
fit_cosine = cos_func(xdata, *parameters)

plt.figure()
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_cosine, '-', label='fit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

## Uncertainty in the fit

The covariance matrix returned by `curve_fit` contains the variance of the fit parameters along the diagonal. The square root of the diagonal elements gives the standard deviation of the fit parameters. This can be used to estimate the uncertainty in the fit.


In [ ]:
uncertainties = np.sqrt(np.diag(covariance))
print("Result of fit")
for name, value, uncertainty in zip(['yscale', 'xscale', 'xshift'], parameters, uncertainties):
    print(f"{name}: {value:.3f} +/- {uncertainty:.3f}")

## Fitting distributions

A $\chi^2$ is used inside the `curve_fit` function to determine the best fit parameters. The $\chi^2$ is defined as
$$
\chi^2 = \sum \left( \frac{y_{\text{data}} - y_{\text{fit}}}{\sigma} \right)^2
$$
where $\sigma$ is the uncertainty in the data. The `curve_fit` function minimizes the $\chi^2$ to determine the best fit parameters.

This corresponds to a maximum likelihood fit if the uncertainties in the data are Gaussian.

More generally, we may want to fit a distribution to data. Thereby, we can use the `scipy.stats` module, which provides a large number of distributions and the minimizan function `scipy.optimize.minimize` to minimize the negative log likelihood.

In [ ]:
gaussdata = np.random.normal(0.2, 1.2, 1000)

In [ ]:
def create_nll(data, dist):
    def nll(params):
        return -np.sum(dist.logpdf(data, *params))

    return nll

In [ ]:
from scipy.stats import norm
from scipy.optimize import minimize

In [ ]:
nll = create_nll(gaussdata, norm)

In [ ]:
result = minimize(nll, [0, 1])

In [ ]:
result.message

In [ ]:
print(f"Best fit parameters: mu = {result.x[0]}, sigma = {result.x[1]}")

## Beyond a Gaussian

The `scipy.stats` module provides a large number of distributions, but is not exhaustive and provides no way to fit distributions that require numerical methods; most importantly, a distribution needs to be normalized by its integral.
SciPy lacks more sophisticated models such as (_or is not optimized for_):
  - multidimensional distributions
  - binned fits
  - products of distributions, convolutions, etc.  (need numerical integration)
  - custom models (only if the integral can be calculated)

As we've seen in our example, the NLL was built from scratch and does not contain any information about the parameters or the model. This can be cumbersome and error-prone. Uncertainties are not provided by default and need to be calculated manually.

But that's fine! SciPy stats is a great tool for simple fits and provides basic distributions with many useful methods.

For more complex fits, other libraries can be used, notably `zfit` for likelihood fits.

# Introduction to zfit

zfit is a model building and fitting library, mostly for likelihood fits and with advanced features for HEP analyses.

It aims to define an API and workflow to be the stable core and allow other libraries to be built on top of it.
Furthermore, it uses JIT compilation and autograd (current backend TensorFlow, future JAX (?)) to significantly speed up the computation.

## Workflow

There are five mostly independent parts in zfit, starting with building a model:

In [ ]:
import zfit

obs = zfit.Space('x', -10, 10)

# create the model
mu = zfit.Parameter("mu", 2.4, -1, 5)
sigma = zfit.Parameter("sigma", 1.3, 0, 5)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

The next step is loading the data (Pandas DataFrames and numpy arrays can be used directly)

In [ ]:
# load the data
data = zfit.Data(obs=obs, data=gaussdata)

The next step involves building the loss or cost function, here an UnbinnedNLL

In [ ]:
# build the loss
nll = zfit.loss.UnbinnedNLL(model=gauss, data=data)

Then, we can create a minimizer and minimize the function (and update the state of the parameters)

In [ ]:
# minimize (20+ interchangeable minimizers available!)
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll).update_params()

Last but not least, the result allows us to perform uncertainty calculations, either an approximation (fast) using `hesse` or a more elaborate profiling using `minos`

In [ ]:
# calculate errors
sym_errors = result.hesse()
asym_errors = result.errors()

In [ ]:
result

Now, let's look at the stages in a bit more detail.

## Data

This component in general plays a minor role in zfit: it is mostly to provide a unified interface for data.

Preprocessing is therefore not part of zfit and should be done beforehand. Python offers many great possibilities to do so (e.g. Pandas).

zfit knows unbinned and binned datasets.

The unbinned `Data` can load data from various sources, such as Numpy, Pandas DataFrame, TensorFlow Tensor and ROOT (using uproot). It is also possible, for convenience, to convert it directly `to_pandas`. The constructors are named `from_numpy`, `from_root` etc.

A `Data` needs not only the data itself but also the observables: the human-readable string identifiers of the axes (corresponding to "columns" of a Pandas DataFrame). It is convenient to define the `Space` not only with the observable but also with a limit: this can directly be re-used as the normalization range in the PDF.

First, let's define our observables

In [ ]:
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import zfit
# znp is a subset of numpy functions with a numpy interface but using actually the zfit backend (currently TF)
import zfit.z.numpy as znp
from zfit import z  # z is the backend, for gradients, JIT etc

In [ ]:
obs = zfit.Space('obs1', -5, 10)
# or more explicitly
obs = zfit.Space(obs='obs1', lower=-5, upper=10, label="Observable one")

In [ ]:
obs.v1.limits

This `Space` has limits and offers the following functionality:
- volume: calculate the volume (e.g. distance between upper and lower)
- inside(): return a boolean Tensor corresponding to whether the value is _inside_ the `Space`
- filter(): filter the input values to only return the one inside

In [ ]:
size_normal = 10_000
data_normal_np = np.random.normal(size=size_normal, scale=2)

data_normal = zfit.Data(data_normal_np, obs=obs)

In [ ]:
data_normal.to_pandas()

The main functionality is
- nevents: attribute that returns the number of events in the object
- space: a `Space` that defines the limits of the data; if outside, the data will be cut (!)
- n_obs: the number of dimensions in the dataset
- with_obs: returns a dataset, possibly a  subset of the dataset with only the given obs
- `with_weights`: returns a dataset with the given weights
- weights: event weights

Using indexing `data[obs]` will return a `Data` object with only the given observables and follows a similar behavior as Pandas DataFrames.
A string, i.e. a single observable,  returns an array, using alist of strings returns a `Data` object with the given observables.

Furthermore, `value` returns a Tensor with shape `(nevents, n_obs)`.

In [ ]:
print(f"We have {data_normal.nevents} events in our dataset with the minimum of {np.min(data_normal.value())}")  # remember! The obs cut out some of the data

In [ ]:
data_normal.n_obs

In [ ]:
data_normal["obs1"]  # indexing Pandas DataFrame style

## Model

Building models is by far the largest part of zfit. We will therefore cover an essential part, the possibility to build custom models, in an extra chapter. Let's start out with the idea that you define your parameters and your observable space; the latter is the expected input data.

PDFs are normalized (over a specified range); this is the main model and is what we are going to use throughout the tutorials.

A PDF is defined by

\begin{align}
\mathrm{PDF}_{f(x)}(x; \theta) = \frac{f(x; \theta)}{\int_{a}^{b} f(x; \theta)}
\end{align}

where a and b define the normalization range (`norm`), over which (by inserting into the above definition) the integral of the PDF is unity.

Let's create a simple Gaussian PDF. We already defined the `Space` for the data before, now we only need the parameters. These are a different objects than a `Space`.

### Parameter
A `Parameter` (there are different kinds actually, more on that later) takes the following arguments as input:
`Parameter(name, initial value[, lower limit, upper limit])` where the limits are highly recommended but not mandatory. Furthermore, `step_size` can be given (which is useful to be around the given uncertainty, e.g. for large yields or small values it can help a lot to set this). Also, a `floating` argument is supported, indicating whether the parameter is allowed to float in the fit or not (just omitting the limits does _not_ make a parameter constant).

Parameters have a unique name. This is served as the identifier for e.g. fit results.

In [ ]:
mu = zfit.Parameter('mu', 1, -3, 3, step_size=0.2)
sigma_num = zfit.Parameter('sigma42', 1, 0.1, 10, floating=False)

Now we have everything to create a Gaussian PDF:

In [ ]:
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma_num)

Since this holds all the parameters and the observables are well-defined, we can retrieve them

In [ ]:
gauss.n_obs  # dimensions

In [ ]:
gauss.obs

In [ ]:
gauss.space

In [ ]:
gauss.norm

As we've seen, the `obs` we defined is the `space` of Gauss: this acts as the default limits whenever needed (e.g. for sampling). `gauss` also has a `norm`, which equals by default as well to the `obs` given, however, we can explicitly change that.

We can also access the parameters of the PDF in two ways, depending on our intention:
either by _name_ (the parameterization name, e.g. `mu` and `sigma`, as defined in the `Gauss`), which is useful if we are interested in the parameter that _describes_ the shape

In [ ]:
gauss.params

or to retrieve all the parameters that the PDF depends on. As this now may sounds trivial, we will see later that models can depend on other models (e.g. sums) and parameters on other parameters. There is one function that automatically retrieves _all_ dependencies, `get_params`. It takes three arguments to filter:
- floating: whether to filter only floating parameters, only non-floating or don't discriminate
- is_yield: if it is a yield, or not a yield, or both
- extract_independent: whether to recursively collect all parameters. This, and the explanation for why independent, can be found later on in the `Simultaneous` tutorial.

Usually, the default is exactly what we want if we look for _all free parameters that this PDF depends on_.

In [ ]:
gauss.get_params()

The difference will also be clear if we e.g. use the same parameter twice:

In [ ]:
gauss_only_mu = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=mu)
print(f"params={gauss_only_mu.params}")
print(f"get_params={gauss_only_mu.get_params()}")

## Functionality

PDFs provide a few useful methods. The main features of a zfit PDF are:

- `pdf`: the normalized value of the PDF. It takes an argument `norm` that can be set to `False`, in which case we retrieve the unnormalized value
- `integrate`: given a certain range, the PDF is integrated. As `pdf`, it takes a `norm` argument that integrates over the unnormalized `pdf` if set to `False`
- `sample`: samples from the pdf and returns a `Data` object

In [ ]:
integral = gauss.integrate(limits=(-1, 3))  # corresponds to 2 sigma integral
np.array(integral)

### Tensors

As we see, many zfit functions return Tensors. Usually, these are `array-Like` objects that resemble numpy arrays close enough that they can be directly used in-place of actual numpy arrays.

If explicitly needed, we can always safely convert them to a numpy array by calling `np.asarray(tensor)`.

More information about the backend can be found in [**Extended tutorial on TensorFlow**](https://zfit-tutorials.readthedocs.io/en/master/tutorials/TensorFlow/HPC_with_TensorFlow.html)

In [ ]:
np.sqrt(integral)  # works out-of-the-box

They also have shapes, dtypes, can be slices etc. So do not convert them except you need it. More on this can be seen in the talk later on about zfit and TensorFlow 2.0.

In [ ]:
sample = gauss.sample(n=1000)  # default space taken as limits
sample

In [ ]:
sample['obs1'][:10]
# sample.value()[:10, 0]  # "value" returns a numpy array-like object

In [ ]:
sample.n_obs

In [ ]:
sample.obs

In [ ]:
sample.to_pandas()

We see that sample returns also a zfit `Data` object with the same space as it was sampled in. This can directly be used e.g.

In [ ]:
probs = gauss.pdf(sample)
probs[:10]

**NOTE**: In case you want to do this repeatedly (e.g. for toy studies), there is a more efficient way (see later on)

## Plotting

So far, we have a dataset and a PDF. Before we go for fitting, we can make a plot:

In [ ]:
def plot_model(model, data, scale=1, plot_data=True):  # we will use scale later on

    nbins = 50

    lower, upper = data.data_range.limit1d
    x = znp.linspace(lower, upper, num=1000)  # np.linspace also works
    y = model.pdf(x) * size_normal / nbins * data.data_range.volume
    y *= scale
    plt.plot(x, y, label='model')
    if plot_data:
        # legacy way, also works
        # data_plot = data.value()[:, 0]  # we could also use the `to_pandas` method
        # plt.hist(data_plot, bins=nbins)
        # modern way
        data_binned = data.to_binned(nbins)
        mplhep.histplot(data_binned, label='data')
    plt.legend()

In [ ]:
plot_model(gauss, data_normal)

We can of course do better (and will see that later on, continuously improve the plots), but this is quite simple and gives us the full power of matplotlib/mplhep.

### Different models

zfit offers a selection of predefined models (and extends with models from zfit-physics that contain physics specific models such as ARGUS shaped models).

In [ ]:
print(zfit.pdf.__all__)

To create a more realistic model, we can build some components for a mass fit with a
- signal component: CrystalBall
- background 1: Exponential
- background 2: Kernel Density Estimation

[**Extended tutorial on different KDEs in zfit**](https://zfit-tutorials.readthedocs.io/en/master/tutorials/components/13%20-%20Kernel%20Density%20Estimation.html)

## Shape fit

In [ ]:
mass_obs = zfit.Space('mass', 0, 1000)

In [ ]:
# Signal component

mu_sig = zfit.Parameter('mu_sig', 400, 100, 600)
sigma_sig = zfit.Parameter('sigma_sig', 50, 1, 100)
alpha_sig = zfit.Parameter('alpha_sig', 200, 100, 400, floating=False)  # won't be used in the fit
npar_sig = zfit.Parameter('n sig', 4, 0.1, 30, floating=False)
signal = zfit.pdf.CrystalBall(obs=mass_obs, mu=mu_sig, sigma=sigma_sig, alpha=alpha_sig, n=npar_sig)

In [ ]:
# background 1
lam = zfit.Parameter('lambda', -0.01, -0.05, -0.0001)
bkg1 = zfit.pdf.Exponential(lam, obs=mass_obs)

In [ ]:
# background 2
bkg2_data = np.random.normal(loc=150, scale=160, size=70_000)
bkg2 = zfit.pdf.KDE1DimGrid(obs=mass_obs, data=bkg2_data)

## Composing models

We can also compose multiple models together. Here we'll stick to one dimensional models, the extension to multiple dimensions is explained in the "custom models tutorial".

Here we will use a `SumPDF`. This takes pdfs and fractions. If we provide n pdfs and:
- n - 1 fracs: the nth fraction will be 1 - sum(fracs)
- n fracs: no normalization attempt is done by `SumPDf`. If the fracs are not implicitly normalized, this can lead to bad fitting
  behavior if there is a degree of freedom too much

[**Extended tutorial on composed models, including multidimensional products**](https://zfit-tutorials.readthedocs.io/en/master/tutorials/TensorFlow/HPC_with_TensorFlow.html)


In [ ]:
sig_frac = zfit.Parameter('sig_frac', 0.3, 0, 1, label="Signal fraction")
bkg1_frac = zfit.Parameter('bkg1_frag', 0.25, 0, 1, label="Bkg1 fraction")
model = zfit.pdf.SumPDF([signal, bkg1, bkg2], [sig_frac, bkg1_frac])

In order to have a corresponding data sample, we can just create one. Since we want to fit to this dataset later on, we will create it with slightly different values.

In [ ]:
with zfit.param.set_values([mu_sig, sigma_sig, sig_frac, bkg1_frac, lam], [370, 34, 0.08, 0.31, -0.0025]):
    data = model.sample(n=10_000)

In [ ]:
params = {mu_sig: 370, sigma_sig: 34, sig_frac: 0.08, bkg1_frac: 0.31, lam: -0.0025}
data = model.sample(n=10_000, params=params)

In [ ]:
plot_model(model, data);

Plotting the components is not difficult now: we can either just plot the pdfs separately (as we still can access them) or in a generalized manner by accessing the `pdfs` attribute:

In [ ]:
def plot_comp_model(model, data):
    for mod, frac in zip(model.pdfs, model.params.values()):
        plot_model(mod, data, scale=frac, plot_data=False)
    plot_model(model, data)

In [ ]:
plot_comp_model(model, data)

Now we can add legends etc. Btw, did you notice that actually, the `frac` params are zfit `Parameters`? But we just used them as if they were Python scalars and it works.

In [ ]:
print(model.params)

### Extended PDFs

So far, we have only looked at normalized PDFs that do contain information about the shape but not about the _absolute_ scale. We can make a PDF extended by adding a yield to it.

The behavior of the new, extended PDF does **NOT change**, any methods we called before will act the same. Only exception, some may require an argument _less_ now. All the methods we used so far will return the same values. What changes is that the flag `model.is_extended` now returns `True`. Furthermore, we have now a few more methods that we can use which would have raised an error before:
- `get_yield`: return the yield parameter (notice that the yield is _not_ added to the shape parameters `params`)
- `ext_{pdf,integrate}`: these methods return the same as the versions used before, however, multiplied by the yield
- `sample` is still the same, but does not _require_ the argument `n` anymore. By default, this will now equal to a _poissonian sampled_ n around the yield.

The `SumPDF` now does not strictly need `fracs` anymore: if _all_ input PDFs are extended, the sum will be as well and use the (normalized) yields as fracs

The preferred way to create an extended PDf is to use `extended=yield` in the PDF constructor.

Alternatively, one can use `extended_pdf = pdf.create_extended(yield)`. (_Since this relies on copying the PDF, which may does not work for different reasons, there is also a `pdf.set_yield(yield)` method that sets the yield in-place. This won't lead to ambiguities, as everything is supposed to work the same._)

In [ ]:
yield_model = zfit.Parameter('yield_model', 10000, 0, 20000, step_size=10)
# model_ext = model.create_extended(yield_model)  # using an existing model

# when creating a new model
model_ext = zfit.pdf.SumPDF([signal, bkg1, bkg2], [sig_frac, bkg1_frac], extended=yield_model)

alternatively, we can create the models as extended and sum them up

In [ ]:
sig_yield = zfit.Parameter('sig_yield', 2000, 0, 10000, step_size=1)
sig_ext = signal.create_extended(sig_yield)

comb_bkg_yield = zfit.Parameter('comb_bkg_yield', 6000, 0, 10000, step_size=1)
comb_bkg_ext = bkg1.create_extended(comb_bkg_yield)

part_reco_yield = zfit.Parameter('part_reco_yield', 2000, 0, 10000, step_size=1)
bkg2.set_yield(
    part_reco_yield)  # unfortunately, `create_extended` does not work here. But no problem, it won't change anything.
part_reco_ext = bkg2

In [ ]:
model_ext_sum = zfit.pdf.SumPDF([sig_ext, comb_bkg_ext, part_reco_ext])

## Loss

A loss combines the model and the data, for example to build a likelihood. Furthermore, it can contain constraints, additions to the likelihood. Currently, if the `Data` has weights, these are automatically taken into account.

In [ ]:
nll_gauss = zfit.loss.UnbinnedNLL(gauss, data_normal)

The loss has several attributes to be transparent to higher level libraries. We can calculate the value of it using `value`.

In [ ]:
nll_gauss.value()

Notice that due to graph building, this will take significantly longer on the first run. Rerun the cell above and it will be way faster.



Furthermore, the loss also provides a possibility to calculate the gradients or, often used, the value and the gradients.

In [ ]:
nll_gauss.gradient()

We can access the data and models (and possible constraints)

In [ ]:
nll_gauss.model

In [ ]:
nll_gauss.data

In [ ]:
nll_gauss.constraints

Similar to the models, we can also get the parameters via `get_params`.

In [ ]:
nll_gauss.get_params()

### Extended loss

More interestingly, we can now build a loss for our composite sum model using the sampled data. Since we created an extended model, we can now also create an extended likelihood, taking into account a Poisson term to match the yield to the number of events.

In [ ]:
nll = zfit.loss.ExtendedUnbinnedNLL(model_ext_sum, data)

In [ ]:
nll.get_params()

## Minimization

While a likelihood is interesting, we usually want to minimize it. Therefore, we can use the minimizers in zfit, most notably `Minuit`, a wrapper around the [iminuit minimizer](https://github.com/scikit-hep/iminuit).

The philosophy is to create a minimizer instance that is mostly _stateless_, e.g. does not remember the position.

Given that iminuit provides us with a very reliable and stable minimizer, it is usually recommended to use this. Others are implemented as well and could easily be wrapped, however, the convergence is usually not as stable.

Minuit has a few options:
- `tol`: the Estimated Distance to Minimum (EDM) criteria for convergence (default 1e-3)
- `verbosity`: between 0 and 10, 5 is normal, 7 is verbose, 10 is maximum
- `gradient`: if True, uses the Minuit numerical gradient instead of the TensorFlow gradient. This is usually more stable for smaller fits; furthermore, the TensorFlow gradient _can_ (experience based) sometimes be wrong. Using `"zfit"` uses the TensorFlow gradient.

In [ ]:
zfit.minimize.__all__

In [ ]:
minimizer = zfit.minimize.Minuit()
# minimizer = zfit.minimize.NLoptLBFGSV1()

### Other minimizers

There are a couple of other minimizers, such as SciPy, NLopt that are wrapped and can be used.

They all provide a highly standardize interface, good initial values and non-ambiguous docstring, i.e. every method has its own docstring.

**When to stop**: a crucial problem is the question of convergence. iminuit has the `EDM`, a comparably reliable metric. Other minimizers in zfit use the same stopping criteria, to be able to compare results, but that is often very inefficienc: many minimizer do not use the Hessian, required for the `EDM`, internally, or do not provide it. This means, the Hessian has to be calculated separately (expensive!). Better stopping criteria ideas are welcome!

In [ ]:
help(zfit.minimize.ScipyNewtonCGV1.__init__)

In [ ]:
print(zfit.minimize.__all__)

In [ ]:
result = minimizer.minimize(nll)

In [ ]:
result

For the minimization, we can call `minimize`, which takes a
- loss as we created above
- optionally: the parameters to minimize

By default, `minimize` uses all the free floating parameters (obtained with `get_params`). We can also explicitly specify which ones to use by giving them (or better, objects that depend on them) to `minimize`; note however that non-floating parameters, even if given explicitly to `minimize` won 't be minimized.

## Fit result

The result of every minimization is stored in a `FitResult`. This is the last stage of the zfit workflow and serves as the interface to other libraries. Its main purpose is to store the values of the fit, to reference to the objects that have been used and to perform (simple) uncertainty estimation.

[**Extended tutorial on the FitResult**](https://zfit-tutorials.readthedocs.io/en/master/tutorials/components/05%20-%20Exploring%20the%20FitResult.html)

In [ ]:
print(result)

This gives an overview over the whole result. Often we're mostly interested in the parameters and their values, which we can access with a `params` attribute.

In [ ]:
print(result.params)

This is a `dict` which stores any knowledge about the parameters and can be accessed by the parameter (object) itself:

In [ ]:
result.params[mu_sig]

'value' is the value at the minimum. To obtain other information about the minimization process, `result` contains more attributes:
- **valid**: whether the result is actually valid or not (most important flag to check)
- fmin: the function minimum
- edm: estimated distance to minimum
- info: contains a lot of information, especially the original information returned by a specific minimizer
- converged: if the fit converged

In [ ]:
result.fmin

Info provides additional information on the minimization, which can be minimizer dependent. For example, if iminuit was used, the instance can be retrieved.

In [ ]:
print(result.info['minuit'])  # the actual instance used in minimization

### Estimating uncertainties

The `FitResult` has mainly two methods to estimate the uncertainty:
- a profile likelihood method (like MINOS)
- Hessian approximation of the likelihood (like HESSE)

When using `Minuit`, this uses (currently) its own implementation. However, zfit has its own implementation, which can be invoked by changing the method name.

Hesse also supports the [corrections for weights](https://inspirehep.net/literature/1762842).

We can explicitly specify which parameters to calculate, by default it does for all.

In [ ]:
result.hesse(name='hesse')

In [ ]:
# result.hesse(method='hesse_np', name='numpy_hesse')
# result.hesse(method='minuit_hesse', name='iminuit_hesse')

We get the result directly returned. This is also added to `result.params` for each parameter and is nicely displayed with an added column

In [ ]:
print(result)

In [ ]:
errors, new_result = result.errors(params=[sig_yield, part_reco_yield, mu_sig],
                                   name='errors')  # just using three, but we could also omit argument -> all used

In [ ]:
errorszfit, new_result_zfit = result.errors(params=[sig_yield, part_reco_yield, mu_sig], method='zfit_errors', name='errorszfit')

In [ ]:
print(errors)

In [ ]:
print(result)

#### What is 'new_result'?

When profiling a likelihood, such as done in the algorithm used in `errors`, a new minimum can be found. If this is the case, this new minimum will be returned, otherwise `new_result` is `None`. Furthermore, the current `result` would be rendered invalid by setting the flag `valid` to `False`. _Note_: this behavior only applies to the zfit internal error estimator.

We can also access the covariance matrix of the parameters

In [ ]:
result.covariance()

### Bonus: serialization

Human-readable serialization is experimentally supported in zfit.
zfit aims to follow the [HS3](https://github.com/hep-statistics-serialization-standard/hep-statistics-serialization-standard) standard, which is currently in development.

[**Extended tutorial on serialization**](https://zfit-tutorials.readthedocs.io/en/latest/tutorials/components/90%20-%20Serialization%20basics.html)

In [ ]:
model.to_dict()

In [ ]:
gauss.to_yaml()

In [ ]:
zfit.hs3.dumps(nll)

### Custom PDFs

zfit allows to build custom PDFs. This is done by inheriting from `BasePDF` and implementing the `_pdf` method. This method should return the unnormalized value of the PDF. The normalization is done automatically by zfit.

Furthemore, it can be used to sample numerically from it.

In [ ]:
import zfit.z.numpy as znp


class CustomPDF2D(zfit.pdf.BasePDF):
    """My custom, 2 dimensional pdf where the axes are: Energy, Momentum."""

    def __init__(
            self,
            param1,
            param2,
            param3,
            obs,
            name="CustomPDF",
            extended=None,
            norm=None,
    ):
        # we can now do complicated stuff here if needed
        # only thing: we have to specify explicitly here what is which parameter
        params = {
            "super_param": param1,  # we can change/compose etc parameters
            "param2": param2,
            "param3": param3,
        }
        super().__init__(obs, params, name=name, extended=None, norm=None)

    @zfit.supports(norm=False)
    def _pdf(self, x, norm, params):
        assert norm is False
        energy = x[0]
        momentum = x[1]
        param1 = params["super_param"]
        param2 = params["param2"]
        param3 = params["param3"]

        # just a fantasy function
        return param1 * znp.cos(energy ** 2) + znp.log(param2 * momentum ** 2) + param3


In [ ]:
import numpy as np

obs = zfit.Space("obs3", -10, 10) * zfit.Space("obs2", -3, 5)
pdf = CustomPDF2D(
    param1=1, param2=2, param3=3, obs=obs
)  # if a Python number is passed, it will be regarded as a constant
sample = pdf.sample(n=1000)
pdf.pdf([[2.0, 2.5], [5.4, 3.2]])

In [ ]:
prod = pdf * gauss  # one dimensional product
prod.n_obs

In [ ]:
prod1d = gauss * signal  # 1dim * 2dim => 3dim product

In [ ]:
prod1d

In [ ]:
binnedsample = sample.to_binned((20, 15))

In [ ]:
import mplhep

In [ ]:
mplhep.hist2dplot(binnedsample)

In [ ]:
binnedsample.to_hist()

In [ ]:
x_part = np.array([2.1, 2.2, 3.2])
pdf.partial_integrate(x=x_part, limits=zfit.Space("obs2", 1, 2))